##  Arbitrage on government treasury bonds <br>
in this code I want to find the best treasury bonds to buy. I seprated bonds to 3 classes based pn their maturity


In [1]:
#import modules we need in this code
import requests as req
import re #regex
import pandas as pd
import numpy as np
import datetime as dt
import khayyam



C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\khayyam\algorithms.py:18: UserWarning: The C extension is not available. Switching to fallback python pure algorithms,so it's about 1000X slower than C implementation of the algorithms.
  warnings.warn(


In [2]:
# use request to scrape ifb.ir , We need bond data
request = req.get('https://www.ifb.ir/ytm.aspx')
request.status_code
html_code = request.text

In [3]:
regex ="<a target=\'_blank\' SymId=\'(.*?)\' href=\'InstrumentsMFI.aspx\?id=(.*?)\' title=\'(.*?)\'>(.*?)</a>\r\n                    </td><td>\r\n                        (.*?)\r\n                    </td><td width=\"(.*?)\">\r\n                        (.*?)\r\n                    </td><td width=\"(.*?)\">\r\n                        (.*?)\r\n                    </td><td width=\"(.*?)\">\r\n                        <div style=\"direction: ltr;\" class=\"\">\r\n                            \r\n                            (.*?)</div>\r\n                    </td><td width=\"(.*?)\">\r\n                        <div style=\"direction: ltr;\" class=\"\">\r\n                            \r\n                            (.*?)</div>\r\n"
bond = re.findall(regex,html_code)
bond[0]

('23935',
 '23935',
 'اسنادخزانه-م13بودجه97-000518',
 'اخزا713',
 '984,000',
 '100',
 '1400-04-16',
 '100',
 '1400-05-18',
 '75',
 '19/53%',
 '75',
 '17/98%')

In [4]:
bond_list = []
for i in range(len(bond)):
    bond_list.append(bond[i][3])

In [5]:
def con(x):
    if ',' in x :
        x= x.replace(',','')
    if '/' in x:
        x= x.replace('/','.')
    if '%' in x:
        x= x.replace('%','')
    return x
def dm(date): #this finction get a string of date and return datetime.date
    y = int(date[:4])
    m = int(date[5:7])
    d= int(date[8:10])
    date = khayyam.JalaliDatetime(y,m,d).todate()
    return date

In [6]:
my_columns = ['symbol','last_price','last_transaction_date','maturity_date','ytm','naive_return']
Bond_df = pd.DataFrame( columns = my_columns)

for i in range(len(bond)):
    Bond_df = Bond_df.append(
                                        pd.Series([
                                            bond[i][3],
                                            int(con(bond[i][4])),
                                            bond[i][6],
                                            bond[i][8],
                                            float(con(bond[i][10]))/100,
                                            float(con(bond[i][12]))/100
                                        ], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [7]:
Bond_df.sort_values(by='ytm', ascending=False).head()

,symbol,last_price,last_transaction_date,maturity_date,ytm,naive_return
8,اخزا805,995990,1400-04-15,1400-04-22,0.2331,0.2099
4,اخزا718,977900,1400-04-16,1400-05-25,0.2262,0.2062
31,اخزا909,684000,1400-04-16,1402-03-16,0.2194,0.2412
21,اخزا820,633800,1400-04-16,1402-08-06,0.2183,0.2502
14,اخزا812,894010,1400-04-16,1400-11-11,0.2173,0.2080


In [8]:
Bond_df['date_to_maturity'] = Bond_df['maturity_date'].apply(lambda x: dm(x)) - Bond_df['last_transaction_date'].apply(lambda x: dm(x))
Bond_df['date_to_maturity'] = Bond_df['date_to_maturity'].apply(lambda x : x.days)


In [9]:
Bond_df.head()

,symbol,last_price,last_transaction_date,maturity_date,ytm,naive_return,date_to_maturity
0,اخزا713,984000,1400-04-16,1400-05-18,0.1953,0.1798,33
1,اخزا723,932000,1400-04-16,1400-08-24,0.2168,0.2033,131
2,اخزا722,994000,1400-04-16,1400-04-28,0.2009,0.1836,12
3,اخزا721,948500,1400-04-16,1400-07-28,0.2018,0.1887,105
4,اخزا718,977900,1400-04-16,1400-05-25,0.2262,0.2062,40


seprate bonds into three classes:<br>
1- bonds which maturite less than 6 month <br>
2- bonds which maturite between 6 month and 1.5 year <br>
3- bonds which matirite more than 1.5 year

In [10]:
def class_bond(x):
    if x <365/2:
        return 'A'
    elif x>365*0.5 and x<365*1.5:
        return 'B'
    else:
        return 'C'

In [11]:
Bond_df['Class_bond'] = Bond_df['date_to_maturity'].apply(lambda x:class_bond(x))

In [12]:
Bond_mean =  Bond_df.groupby('Class_bond')['ytm'].aggregate("mean").reset_index()
Bond_mean= Bond_mean.rename(columns = {'ytm':'ytm_mean'})
Bond_mean

,Class_bond,ytm_mean
0,A,0.205446
1,B,0.216125
2,C,0.211436


In [13]:
Bond_df = pd.merge(Bond_df,Bond_mean,on = 'Class_bond', how = 'left')

In [14]:
Bond_df.head()

,symbol,last_price,last_transaction_date,maturity_date,ytm,naive_return,date_to_maturity,Class_bond,ytm_mean
0,اخزا713,984000,1400-04-16,1400-05-18,0.1953,0.1798,33,A,0.205446
1,اخزا723,932000,1400-04-16,1400-08-24,0.2168,0.2033,131,A,0.205446
2,اخزا722,994000,1400-04-16,1400-04-28,0.2009,0.1836,12,A,0.205446
3,اخزا721,948500,1400-04-16,1400-07-28,0.2018,0.1887,105,A,0.205446
4,اخزا718,977900,1400-04-16,1400-05-25,0.2262,0.2062,40,A,0.205446


In [15]:
Bond_df['arbitrage'] = Bond_df['ytm'] - Bond_df['ytm_mean']

# Arbitrage table

In [16]:
Bond_df.sort_values('arbitrage',ascending=False)

,symbol,last_price,last_transaction_date,maturity_date,ytm,naive_return,date_to_maturity,Class_bond,ytm_mean,arbitrage
8,اخزا805,995990,1400-04-15,1400-04-22,0.2331,0.2099,7,A,0.205446,0.027654
4,اخزا718,977900,1400-04-16,1400-05-25,0.2262,0.2062,40,A,0.205446,0.020754
5,اخزا811,907573,1400-04-16,1400-10-13,0.2173,0.2065,180,A,0.205446,0.011854
1,اخزا723,932000,1400-04-16,1400-08-24,0.2168,0.2033,131,A,0.205446,0.011354
31,اخزا909,684000,1400-04-16,1402-03-16,0.2194,0.2412,699,C,0.211436,0.007964
11,اخزا808,936500,1400-04-16,1400-08-17,0.2130,0.1996,124,A,0.205446,0.007554
12,اخزا809,919000,1400-04-16,1400-09-23,0.2125,0.2011,160,A,0.205446,0.007054
21,اخزا820,633800,1400-04-16,1402-08-06,0.2183,0.2502,843,C,0.211436,0.006864
26,اخزا904,721101,1400-04-16,1401-12-15,0.2173,0.2326,607,C,0.211436,0.005864
24,اخزا902,743870,1400-04-16,1401-10-19,0.2165,0.2281,551,C,0.211436,0.005064
